네이버 전처리
1. DB 구축을 위한 primary 키 삽입 string(위도, 경도)
2. 중복제거(사업장명, 위도, 경도)
3. primary key nan제거

In [44]:
import pandas as pd
import numpy as np
import re
import os

In [45]:
naver_df = pd.read_csv('../../naver_crw/DAOU/test_large_out_v3.csv', index_col=0, encoding='euc-kr')

In [46]:
print(naver_df.info())
naver_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   가까운 지하철역                5774 non-null   object 
 6   지하철역과의 거리               6664 non-null   float64
 7   도보시간                    6664 non-null   float64
 8   위도                      5774 non-null   float64
 9   경도                      5774 non-null   float64
 10  별점                      310 non-null    float64
 11  방문자 리뷰수                 685 non-null    object 
 12  블로그 리뷰수                 513 non-null    object 
 13  한실수                     5434 non-null   float64
 14  양실수                     5347 non-null   

In [114]:
# 프라이머리 키 생성
for i in range(naver_df.shape[0]):
    if pd.isnull(naver_df['위도'][i]) and pd.isnull(naver_df['경도'][i]):
        naver_df['naver_pk'][i] = np.nan
    else:        
        naver_df['naver_pk'][i] = f"{naver_df['위도'][i]:.10f}, {naver_df['경도'][i]:.10f}"   



In [140]:
# .. 위경도 프라이머리키로 중복 제거
# 중복 개수 확인
df_temp1 = naver_df.groupby(['naver_pk'])['도로명주소'].count().reset_index(name='호실개수')
duplicate_name = df_temp1[df_temp1.호실개수 >= 2] # 21개
duplicate_name
num_nankey = len(naver_df[pd.isna(naver_df['naver_pk'])])
print(f'중복 개수 {len(duplicate_name)}, nan 개수 {num_nankey}')

naver_df_p1 = naver_df.drop_duplicates(['naver_pk'])    # 6664 --> 6383
print(len(duplicate_name), naver_df_p1.shape[0], naver_df.shape[0])

중복 개수 142, nan 개수 890
142 5439 6664


In [141]:
naver_df_p1.to_csv("../DAOU/naver.csv", encoding='euc-kr', index=False)